# Payoff d'une iron butterfly sur SPY

Iron butterfly (short put/call au centre, ailes longues) : payoff crédit, P&L net (prime BS) et points morts.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
from pricing import fetch_spy_history, view_iron_butterfly, payoff_iron_butterfly
plt.style.use("seaborn-v0_8-darkgrid")

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])

k_center = spot_ref
k_put_long = spot_ref * 0.95
k_call_long = spot_ref * 1.05
view_static = view_iron_butterfly(spot_ref, k_put_long, k_center, k_call_long)
premium = view_static["premium"]
be_low, be_high = view_static["breakevens"]
credit = -premium


## Payoff interactif
Curseurs pour S_0, S_T, strike central et ailes. Crédit/points morts recalculés dynamiquement.


In [ ]:
spot0_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_0')
spotT_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_T')
slider_put_long = widgets.FloatSlider(value=k_put_long, min=spot_ref*0.8, max=spot_ref*1.0, step=1.0, description='K put long')
slider_center = widgets.FloatSlider(value=k_center, min=spot_ref*0.9, max=spot_ref*1.1, step=1.0, description='K center')
slider_call_long = widgets.FloatSlider(value=k_call_long, min=spot_ref*1.0, max=spot_ref*1.2, step=1.0, description='K call long')
output = widgets.Output()

def _update_payoff(change=None):
    with output:
        output.clear_output()
        s0 = spot0_slider.value
        sT = spotT_slider.value
        k_put_val = slider_put_long.value
        k_center_val = slider_center.value
        k_call_val = slider_call_long.value

        view_dyn = view_iron_butterfly(s0, k_put_val, k_center_val, k_call_val)
        premium_dyn = view_dyn["premium"]
        credit_dyn = -premium_dyn
        be_low, be_high = view_dyn["breakevens"]
        payoff_T = float(payoff_iron_butterfly(sT, k_put_val, k_center_val, k_call_val))
        pnl_T = payoff_T - premium_dyn

        fig, ax = plt.subplots(figsize=(7, 4))
        ax.plot(view_dyn["s_grid"], view_dyn["payoff"], label='Payoff brut')
        ax.plot(view_dyn["s_grid"], view_dyn["pnl"], label='P&L net', color='darkorange')
        for k, style, label in [
            (k_put_val, ':', f'K put long = {k_put_val:.2f}'),
            (k_center_val, '--', f'K center = {k_center_val:.2f}'),
            (k_call_val, ':', f'K call long = {k_call_val:.2f}'),
        ]:
            ax.axvline(k, color='gray', linestyle=style, label=label)
        ax.axvline(be_low, color='forestgreen', linestyle=':', label=f'Point mort bas ≈ {be_low:.2f}')
        ax.axvline(be_high, color='forestgreen', linestyle='--', label=f'Point mort haut ≈ {be_high:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S_0 = {s0:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.scatter([sT], [pnl_T], color='gold', zorder=5, label=f'P&L à S_T = {pnl_T:.2f}')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.legend(loc='lower right')
        ax.set_title("Payoff et P&L de l'iron butterfly")
        plt.show()
        plt.close(fig)

        msg = f"""**S_0 = {s0:.2f}**, **S_T = {sT:.2f}**

- K put long = {k_put_val:.2f}
- K center = {k_center_val:.2f}
- K call long = {k_call_val:.2f}
- Crédit net (BS) = {credit_dyn:.4f}
- Payoff à l'expiration = {payoff_T:.4f}
- P&L net = {pnl_T:.4f}
- Points morts ≈ {be_low:.2f} / {be_high:.2f}"""
        display(Markdown(msg))

for sl in (spot0_slider, spotT_slider, slider_put_long, slider_center, slider_call_long):
    sl.observe(_update_payoff, names='value')

_update_payoff()
display(widgets.VBox([spot0_slider, spotT_slider, slider_put_long, slider_center, slider_call_long, output]))
